In [60]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_extraction.text import TfidfVectorizer

In [61]:
dataset = pd.read_csv("mail_data.csv")

In [62]:
dataset.tail()

,Category,Message
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will ü b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...
5571,ham,Rofl. Its true to its name


In [63]:
from sklearn.preprocessing import LabelEncoder

In [64]:
en = LabelEncoder()
dataset["Category"] = en.fit_transform(dataset["Category"],)

In [65]:
dataset.isnull().sum()

Category    0
Message     0
dtype: int64

In [66]:
dataset.shape

(5572, 2)

In [67]:
x = dataset["Message"]

In [68]:
x.shape

(5572,)

In [69]:
y = dataset["Category"]
y.shape

(5572,)

In [70]:
# import matplotlib.pyplot as plt
# plt.scatter(x,y)

ham ---> 0

spam --> 1

In [71]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size = 0.3,random_state = 34)

In [72]:
feature_extraction = TfidfVectorizer(min_df = 1,stop_words= 'english',lowercase = 'True')
x_train_features = feature_extraction.fit_transform(x_train)
x_test_features = feature_extraction.transform(x_test)

y_train = y_train.astype('int')
y_test = y_test.astype('int')

In [73]:
x_train_features

<3900x6923 sparse matrix of type '<class 'numpy.float64'>'
	with 30619 stored elements in Compressed Sparse Row format>

In [74]:
x_test_features

<1672x6923 sparse matrix of type '<class 'numpy.float64'>'
	with 11217 stored elements in Compressed Sparse Row format>

In [75]:
model_1 = RandomForestClassifier()
model_1.fit(x_train_features,y_train)

RandomForestClassifier()

In [76]:
from sklearn.metrics import accuracy_score
model_1.score(x_test_features,y_test)
prediction_on_training_data = model_1.predict(x_train_features)
accuracy_on_training_data = accuracy_score(y_train, prediction_on_training_data)

In [77]:
accuracy_on_training_data

1.0

In [78]:
model_2 = LogisticRegression()
model_2.fit(x_train_features,y_train)
model_2.score(x_test_features,y_test)

0.9515550239234449

In [79]:
model_3 =  DecisionTreeClassifier()
model_3.fit(x_train_features,y_train)
model_3.score(x_test_features,y_test)

0.9593301435406698

In [80]:
from sklearn.svm import SVC
model_4 = SVC()
model_4.fit(x_train_features,y_train)
model_4.score(x_test_features,y_test)

0.9748803827751196

In [81]:
model_4.score(x_test_features,y_test)

0.9748803827751196

In [82]:
from sklearn.model_selection import GridSearchCV
  
# defining parameter range
param_grid = {'C': [0.1, 1, 10, 100, 1000], 
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
              'kernel': ['rbf']} 
  
grid = GridSearchCV(SVC(), param_grid, refit = True, verbose = 3)
  
# fitting the model for grid search
grid.fit(x_train_features, y_train)

Fitting 5 folds for each of 25 candidates, totalling 125 fits
[CV 1/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.867 total time=   0.3s
[CV 2/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.867 total time=   0.3s
[CV 3/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.867 total time=   0.3s
[CV 4/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.867 total time=   0.3s
[CV 5/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.865 total time=   0.3s
[CV 1/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.867 total time=   0.2s
[CV 2/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.867 total time=   0.2s
[CV 3/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.867 total time=   0.2s
[CV 4/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.867 total time=   0.2s
[CV 5/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.865 total time=   0.2s
[CV 1/5] END .....C=0.1, gamma=0.01, kernel=rbf;, score=0.867 total time=   0.1s
[CV 2/5] END .....C=0.1, gamma=0.01, kernel=rbf

GridSearchCV(estimator=SVC(),
             param_grid={'C': [0.1, 1, 10, 100, 1000],
                         'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
                         'kernel': ['rbf']},
             verbose=3)

In [83]:
# print best parameter after tuning
print(grid.best_params_)

# print how our model looks after hyper-parameter tuning
print(grid.best_estimator_)


{'C': 10, 'gamma': 0.1, 'kernel': 'rbf'}
SVC(C=10, gamma=0.1)


In [84]:
model_5 = SVC(C=10, gamma=0.1,kernel='rbf')
model_5.fit(x_train_features,y_train)
model_5.score(x_test_features,y_test)*100

98.56459330143541

In [85]:
pred = model_5.predict(x_test_features)

In [86]:
print(pred)

[0 0 1 ... 1 0 0]


In [87]:
import joblib
joblib.dump(model_5,"Email_Spam_Detection_model.joblib")
joblib.dump(feature_extraction,"vectorizer.joblib")

['vectorizer.joblib']

accuracy = 98.56459330143541

In [89]:
import joblib
model = joblib.load("Email_Spam_Detection_model.joblib")
vectorizer = joblib.load("vectorizer.joblib")

input_data = ["Festive Season Sale Savings-TODAY ONLY! ($60 instead of $180)"]
input_data_features = vectorizer.transform(input_data)
prediction = model.predict(input_data_features)


if (prediction[0]==0):
  print('Ham mail')

else:
  print('Spam mail')

Ham mail
